In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
organizations_mlg_ulb_creditcardfraud_path = kagglehub.dataset_download('organizations/mlg-ulb/creditcardfraud')

print('Data source import complete.')


In [ ]:
!pip install pykan torch-optimizer shap lime --quiet
import pandas as pd
import sklearn.metrics as metrique
from pandas import Series
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn.functional as F
import kan

In [ ]:
data = pd.read_csv(r"/kaggle/input/creditcardfraud/creditcard.csv")
data

In [ ]:
from imblearn.over_sampling import ADASYN
oversample = ADASYN()
X = data.drop(['Class'], axis = 'columns')
y = data['Class']
X, y = oversample.fit_resample(X, data['Class'])

In [ ]:
from collections import Counter

counter = Counter(y)
print(counter)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(type(y_train))
print(type(y_test))

y_train = y_train.values if isinstance(y_train, pd.Series) else y_train
y_test = y_test.values if isinstance(y_test, pd.Series) else y_test

X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(-1)
y_test = torch.tensor(y_test, dtype=torch.long)


train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)


for X_batch, y_batch in train_loader:
    print(f"Batch X shape: {X_batch.shape}, Batch y shape: {y_batch.shape}")
    break

In [ ]:
class LinearAttention(nn.Module):
    def __init__(self):
        super(LinearAttention, self).__init__()
        self.eps = 1e-6

    def elu_feature_map(self, x):
        return F.elu(x) + 1

    def forward(self, Q, K, V):
        Q = self.elu_feature_map(Q)
        K = self.elu_feature_map(K)
        KV = torch.einsum("nsd,nsd->ns", K, V)
               Z = 1/(torch.einsum("nld,nd->nl", Q, K.sum(dim=1))+self.eps)
                V = torch.einsum("nld,ns,nl->nd", Q, KV, Z)
        return V.contiguous()

In [ ]:
class LTACNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LTACNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.attention_layer = LinearAttention()
        self.fc = nn.Linear(241, 1)


    def forward(self, x):

        x_cnn = x.permute(0, 2, 1)
        out_cnn = self.cnn(x_cnn)
        out_cnn = out_cnn.reshape(out_cnn.size(0), -1)


        x_lstm = x
        out_lstm, _ = self.lstm(x_lstm)


        out_lstm_last = out_lstm[:, -1, :] if out_lstm.dim() == 3 else out_lstm


        combined = torch.cat((out_cnn, out_lstm_last), dim=1)
        attention_combined = self.attention_layer(combined.unsqueeze(1), combined.unsqueeze(1), combined.unsqueeze(1))



        out = self.fc(attention_combined)
        out = torch.sigmoid(out)

        return out

In [ ]:
from torch.optim import Adam
import torch.optim as optim
input_size = X_train.shape[-1]
hidden_size = 128
num_layers = 2
num_classes = len(np.unique(y_train))

LTACNN = LTACNN(input_size=input_size, hidden_size=1, num_classes=2)

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch_optimizer import Ranger
from sklearn.metrics import roc_auc_score, precision_score, recall_score
import numpy as np
import time

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_metrics(performance_metrics, epochs):
    sns.set(style="whitegrid")
    metrics = ['loss', 'roc_auc', 'precision', 'recall','test_roc_auc', 'test_recall', 'test_precision']

    for metric in metrics:
        plt.figure(figsize=(10, 6))
        for model_name, history in performance_metrics.items():
            plt.plot(range(1, epochs+1), history[metric], label=model_name)

        plt.title(f'Comparison of {metric.capitalize()} Across Epochs')
        plt.xlabel('Epoch')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.tight_layout()
        plt.show()

In [ ]:
from kan import *
class kan_model(nn.Module):
    def __init__(self):
        super(kan_model, self).__init__()
        self.kan = KAN(width=[30, 60, 120, 241, 1], grid=6, k=8, seed=42)
        self.kan.speed()
        self.kan.save_act = True

    def forward(self, x):
        x = x.squeeze(-1)
        out = self.kan(x)
        return torch.sigmoid(out)
kan_model = kan_model()

In [ ]:
def calculate_metrics(y_true, y_pred_probs):
    """Calculate ROC-AUC, Precision, and Recall from ground truth labels and predicted probabilities."""
    roc_auc = roc_auc_score(y_true, y_pred_probs)
    binary_preds = [1 if p >= 0.5 else 0 for p in y_pred_probs]
    precision = precision_score(y_true, binary_preds)
    recall = recall_score(y_true, binary_preds)
    return roc_auc, precision, recall

# Training function
def train_model_on_gpu(model, train_loader, test_loader=None, epochs=10, device="cuda"):
    model.train()
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001) 
    criterion = nn.BCELoss()  

    history = {'roc_auc': [], 'precision': [], 'recall': [], 'loss': [], 'test_roc_auc': [], 'test_precision': [], 'test_recall': []}

    for epoch in range(epochs):
        all_labels, all_preds, epoch_losses = [], [], []

        for x, y in train_loader:
            x, y = x.to(device), y.to(device).float()  

            optimizer.zero_grad()
            y_pred = model(x)  # Get predictions
            y_pred = y_pred.squeeze()  

            loss = criterion(y_pred, y)  
            loss.backward()
            optimizer.step()

            epoch_losses.append(loss.item())
            all_labels.extend(y.cpu().numpy())
            all_preds.extend(y_pred.detach().cpu().numpy())

        avg_loss = np.mean(epoch_losses)
        history['loss'].append(avg_loss)
        roc_auc, precision, recall = calculate_metrics(all_labels, all_preds)
        history['roc_auc'].append(roc_auc)
        history['precision'].append(precision)
        history['recall'].append(recall)

        print(f"[{model.__class__.__name__} | Epoch {epoch+1}/{epochs}] Loss: {avg_loss:.4f} | ROC-AUC: {roc_auc:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")

        # Evaluate on test data
        if test_loader:
            model.eval()
            test_labels, test_preds = [], []

            with torch.no_grad():
                for x_test, y_test in test_loader:
                    x_test, y_test = x_test.to(device), y_test.to(device).float()
                    y_test_pred = model(x_test)

                    y_test_pred = y_test_pred.squeeze()
                    test_labels.extend(y_test.cpu().numpy())
                    test_preds.extend(y_test_pred.cpu().numpy())

            test_roc_auc, test_precision, test_recall = calculate_metrics(test_labels, test_preds)
            history['test_roc_auc'].append(test_roc_auc)
            history['test_precision'].append(test_precision)
            history['test_recall'].append(test_recall)

            print(f"[{model.__class__.__name__} | Test | Epoch {epoch+1}/{epochs}] ROC-AUC: {test_roc_auc:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}")
            model.train()

    return history


def train(models, train_loader, test_loader=None, epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    result_dict = {}

    for model in models:
        print(f"Training model: {model.__class__.__name__}")
        history = train_model_on_gpu(model, train_loader, test_loader, epochs, device)
        result_dict[model.__class__.__name__] = history

    return result_dict

In [ ]:
metrics = train([kan_model, LTACNN], train_loader,test_loader, 15)

In [ ]:
plot_metrics(metrics, 15)

In [ ]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score
from torch import nn
from torch.utils.data import DataLoader

def evaluate(models, test_loader: DataLoader, device=None):
    """Evaluates multiple models on a test set and returns performance metrics."""

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cuda")

    performance_metrics = {}

    for model in models:
        print(f"Evaluating model: {model.__class__.__name__}")
        model.to(device)
        model.eval()

        history = {'roc_auc': [], 'precision': [], 'recall': [], 'loss': []}

        all_labels = []
        all_preds = []
        test_losses = []


        criterion = nn.BCELoss()

        with torch.no_grad():
            for x, y in test_loader:
                x = x.to(device)
                y = y.to(device)


                y_pred = model(x)

                y_pred = y_pred.squeeze()


                loss = criterion(y_pred, y.float())
                test_losses.append(loss.item())


                all_labels.extend(y.cpu().numpy())
                all_preds.extend(y_pred.cpu().numpy())


        avg_loss = np.mean(test_losses)
        history['loss'].append(avg_loss)

        roc_auc = roc_auc_score(all_labels, all_preds)
        history['roc_auc'].append(roc_auc)


        binary_preds = [1 if p >= 0.5 else 0 for p in all_preds]
        precision = precision_score(all_labels, binary_preds)
        recall = recall_score(all_labels, binary_preds)

        history['precision'].append(precision)
        history['recall'].append(recall)

        print(f"[{model.__class__.__name__} | Test] Loss: {avg_loss:.4f}, ROC-AUC: {roc_auc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

        performance_metrics[model.__class__.__name__] = history

    return performance_metrics


In [ ]:
test_metrics = evaluate([LTACNN, kan_model], test_loader)

In [ ]:
import torch
import shap
import lime.lime_tabular
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, wilcoxon


models = {
    "LTACNN": LTACNN,
    "KAN": kan_model,

}


num_samples = 1000
random_indices_train = torch.randperm(X_train.shape[0])[:num_samples]
random_indices_test = torch.randperm(X_test.shape[0])[:num_samples]

X_train_sample = X_train[random_indices_train].clone().detach().cpu()
X_test_sample = X_test[random_indices_test].clone().detach().cpu()


X_train_np = X_train_sample.numpy()
X_test_np = X_test_sample.numpy()


shap_values_dict = {}

for name, model in models.items():
    model.to("cpu").eval()


    explainer = shap.DeepExplainer(model, X_train_sample)
    shap_values = explainer.shap_values(X_test_sample, check_additivity=False)
    X_test_flattened = X_test_sample.reshape(X_test_sample.shape[0], -1)


    shap_values_flattened = np.array(shap_values).reshape(shap_values.shape[0], -1)


    shap_values_dict[name] = shap_values_flattened


    shap.summary_plot(shap_values_flattened, X_test_flattened.numpy(), show=False)
    plt.title(f"SHAP Summary Plot - {name}")
    plt.show()


shap_df = pd.DataFrame({
    "Feature": [f"F{i}" for i in range(X_test_np.shape[1])]
})

for name in models.keys():
    shap_df[name] = np.mean(np.abs(shap_values_dict[name]), axis=0)

print(shap_df.head())

In [ ]:
print("X_train_np shape:", X_train_np.shape)
num_features = X_train_np.shape[1]
feature_names = [f"F{i}" for i in range(num_features)]
print("Number of features:", num_features)
print("Feature names:", feature_names)
X_train_np = X_train_np.reshape(num_samples, -1)
print("X_train_np shape:", X_train_np.shape)

In [ ]:

explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_np, feature_names=shap_df["Feature"].values, mode="regression")


lime_values_dict = {}
X_test_np = X_test_np.reshape(num_samples, -1)
for name, model in models.items():
    exp = explainer.explain_instance(
        X_test_np[0],

        lambda x: model(
            torch.tensor(x, dtype=torch.float32)
                 .unsqueeze(-1)
        ).detach().cpu().numpy(),
        num_features=X_train_np.shape[1]
    )


    lime_values_dict[name] = dict(exp.as_list())


lime_df = pd.DataFrame(lime_values_dict).fillna(0)
lime_df["Feature"] = shap_df["Feature"]

print(lime_df.head())


shap_melted = shap_df.melt(id_vars="Feature", var_name="Model", value_name="SHAP Value")
lime_melted = lime_df.melt(id_vars="Feature", var_name="Model", value_name="LIME Value")
lime_df = lime_df.reset_index().rename(columns={"index": "Rule"})
lime_melted_rules = lime_df.melt(id_vars="Rule", var_name="Model", value_name="LIME Value")

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x="Feature", y="SHAP Value", hue="Model", data=shap_melted)
plt.xticks(rotation=45)
plt.title("SHAP Feature Importance Across Models")
plt.show()

In [ ]:
print("shap_df:")
print(shap_df.head())
print("lime_df:")
print(lime_df.head())
print("lime_melted:")
print(lime_melted.head())
print("lime_melted columns:", lime_melted.columns)
print("lime_melted shape:", lime_melted.shape)

In [ ]:
plt.figure(figsize=(24, 6))
sns.barplot(x="Rule", y="LIME Value", hue="Model", data=lime_melted_rules)
plt.xticks(rotation=45)
plt.title("LIME Rule Importance Across Models")
plt.show()

In [ ]:
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt




def extract_feature(rule):

    m = re.search(r'(F\d+)', rule)
    return m.group(1) if m else None

lime_df['Feature'] = lime_df['Rule'].apply(extract_feature)

print("lime_df after extracting 'Feature':")
print(lime_df.head())


numeric_cols = ['LTACNN', 'kan_model']

lime_df_grouped = lime_df.groupby('Feature')[numeric_cols].mean().reset_index()

print("\nAggregated LIME DataFrame (lime_df_grouped):")
print(lime_df_grouped.head())

merged_df = shap_df.set_index("Feature").join(
    lime_df_grouped.set_index("Feature"), lsuffix="_SHAP", rsuffix="_LIME"
)


merged_df_numeric = merged_df.select_dtypes(include=["number"])


print("\nMerged DataFrame (numeric only):")
print(merged_df_numeric.head())


corr_matrix = merged_df_numeric.corr(method="spearman")

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Spearman Correlation of SHAP & LIME Across Models")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

In [ ]:
for m1, m2 in [("LTACNN", "kan_model")]:
    print(f"Wilcoxon Test (SHAP {m1} vs {m2}):", wilcoxon(shap_df[m1], shap_df[m2]))
    print(f"Wilcoxon Test (LIME {m1} vs {m2}):", wilcoxon(lime_df[m1], lime_df[m2]))